# DATASCI 503, Group Work 6: Splines and GAMs

**Instructions:** During lab section, and afterward as necessary, you will collaborate in two-person teams (assigned by the GSI) to complete the problems that are interspersed below. The GSI will help individual teams encountering difficulty, make announcements addressing common issues, and help ensure progress for all teams. During lab, feel free to flag down your GSI to ask questions at any point!

### Requirements

For this lab we will need to install the `pygam` library.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pygam import LinearGAM, LogisticGAM, f, s
from pygam.datasets import default
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder,
    PolynomialFeatures,
    SplineTransformer,
    StandardScaler,
)

### `sklearn` [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

''A sequence of data transformers with an optional final predictor.  
Pipeline allows you to sequentially apply a list of transformers to preprocess the data and, if desired, conclude the sequence with a final predictor for predictive modeling.  
Intermediate steps of the pipeline must be transformers, that is, they must implement fit and transform methods. The final estimator only needs to implement fit. The transformers in the pipeline can be cached using memory argument.  
The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.''

In [ ]:
# Re-defining the data
np.random.seed(100)
x = np.linspace(0, 10, 30)
y = 2 * np.sin(0.5 * x) + np.cos(x) + np.random.randn(30)
X = x.reshape(-1, 1)  # Reshaping x for the models

# scatter plot of data
plt.scatter(X, y, label="Data")
plt.legend()
plt.show()

In [ ]:
# Defining the pipeline with polynomial features
poly_pipeline = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=5)),
        ("scaling", StandardScaler()),
        ("linear_regression", LinearRegression()),
    ]
)

# Fitting the model
poly_pipeline.fit(X, y)
# Predicting over the input range
X_test = np.linspace(0, 10, 200).reshape(-1, 1)
y_poly_pred = poly_pipeline.predict(X_test)

plt.scatter(X, y, label="Data")
plt.plot(X_test, y_poly_pred, label="Polynomial Fit", color="red")
plt.title("Polynomial Feature Fit")
plt.legend()
plt.show()

Let's check what's inside a `Pipeline` object

In [ ]:
poly_pipeline

In [ ]:
degrees = range(0, 12, 2)

plt.figure(figsize=[10, 6])
plt.scatter(X, y, label="Data")

for degree in degrees:
    poly_pipeline = Pipeline(
        [
            ("poly", PolynomialFeatures(degree=degree)),
            ("scaling", StandardScaler()),
            ("linear_regression", LinearRegression()),
        ]
    )

    # Fitting the model
    poly_pipeline.fit(X, y)
    # Predicting over the input range
    X_test = np.linspace(0, 10, 200).reshape(-1, 1)
    y_poly_pred = poly_pipeline.predict(X_test)

    # Plotting
    plt.plot(X_test, y_poly_pred, label=f"Polynomial Fit degree:{degree}")

plt.title("Polynomial Feature Fits for Different Degrees")
plt.legend()
plt.show()

### Cubic [Splines](https://en.wikipedia.org/wiki/Spline_(mathematics))

Consider a simple setup where we have data $\{(x_i,y_i)\}_{i=1}^n$, and $x_i\in \mathbb{R}^1$. We want to model the non-linear relationship between $y$ and $x$. Cubic spline is a method that fits a piece-wise cubic polynomial on contiguous intervals of $x$. Suppose that the minimum and maximum of all $x_i$ are $\xi_0,\xi_{K+1}$, respectively. There are $K$ knots in between that serve as the endpoints of the $K+1$ intervals of $x$: $\xi_1 < \xi_2 < \dots < \xi_K$. On every interval $[\xi_{k-1}, \xi_{k}]$ we fit a cubic polynomial $s_k(x)$ for $y$. Additionally, on each of the knot, we constrain that:

- Continuity: $s_k(\xi_{k}) = s_{k+1}(\xi_{k})$
- First-order continuity: $s_k^{\prime}(\xi_{k}) = s_{k+1}^{\prime}(\xi_{k})$
- Second-order continuity: $s_k^{\prime\prime}(\xi_{k}) = s_{k+1}^{\prime\prime}(\xi_{k})$

On every of the $K+1$ intervals, we have a cubic polynomial with 4 parameters, but on every of the $K$ knots we have the 3 constraints above. In aggregate, we have $4(K+1)−3K=K+4$ free parameters for cubic spline.

$$
s(x) = \beta_0 h_0(x) + \beta_1 h_1(x) + \dots + \beta_{K+3} h_{K+3}(x)
$$

where $h_{j}(x) = x^{j}, j = 0, 1, 2, 3$, and $h_{3+l}(x) = (x-\xi_l)_{+}^3, l=1,2,\dots,K$.

Documentation: [Spline Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.SplineTransformer.html)

**So how many knots?**

In general, it turns out that the optimal number of knots is data dependent, and the rule-of-thumb of choosing knot location is to put more knots in the interval where function is varying a lot, and put fewer knots in the interval where the function is very smooth and can be approximated well by very few cubic polynomial. In practice, this requires you to do **exploratory data analysis**! A spline method with automatic knot selection called *smoothing spline* in the next section.

In [ ]:
# Creating the pipeline with SplineTransformer and LinearRegression
spline_pipeline = Pipeline(
    [("spline", SplineTransformer(n_knots=4, degree=3)), ("linear_regression", LinearRegression())]
)

# Fitting the pipeline to the data
spline_pipeline.fit(X, y)

# Generating a range of values for plotting the spline
X_plot = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
y_spline = spline_pipeline.predict(X_plot)

# Plotting the original data and the spline fit
plt.figure(figsize=(10, 6))
plt.scatter(X, y, facecolor="gray", label="Data")
plt.plot(X_plot, y_spline, label="Spline fit", color="dodgerblue")
plt.title("Spline Fitting Example")
plt.legend()
plt.show()

In [ ]:
# Extracting the spline transformer from the pipeline
spline_transformer = spline_pipeline.named_steps["spline"]
X_splines = spline_transformer.transform(X)
# Generating a range of values for plotting the spline
X_plot = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
X_plot_splines = spline_transformer.transform(X_plot)
# Extracting the knot positions
knots = spline_transformer.bsplines_[0].t[3:7]

# Plotting the original data
plt.figure(figsize=(10, 6))
plt.scatter(X, y, facecolor="gray", label="Data")

# Plotting the splines
for i in range(X_splines.shape[1]):
    plt.plot(X_plot, X_plot_splines[:, i], label=f"Spline {i + 1}", linestyle="--")

# Plotting the spline fit
y_spline = spline_pipeline.predict(X_plot)
plt.plot(X_plot, y_spline, label="Spline fit", color="dodgerblue", linewidth=2)


# Adding markers at the knot positions
plt.scatter(knots, np.zeros_like(knots), color="red", marker="x", s=100, label="Knots", zorder=5)

plt.title("Spline Features and Fitting Example")
plt.legend()
plt.show()

### Smoothing Splines

Smoothing splines are particularly useful in situations where you want to fit a curve through data points in a way that balances the fit's closeness to the data points against the smoothness of the fit. In other words, we want to fit a curve that:  

- Minimizes Residual Squared Error,
- Is Smooth

$$
\sum_{i=1}^n(y_i - g(x_i))^2 + \lambda \int (g^{(m)}(t))^2 dt
$$
where $\lambda \geq 0$ is a *tuning parameter* and $g^{(m)}$ is the $m$-th derivative

The function $g(x)$ that minimizes this objective has special properties:

- It is a piecewise polynomial of degree $2m-1$,
- It has knots at $x_1, x_2, \cdots, x_n$, and
- In the region outside the knots at the extremes, it is a polynomial of degree $m-1$.


**Think About:** How does $\lambda$ control the bias-variance tradeoff of the smoothing spline?


**Think About:** What are the limits of the integral defining the objective?

In order to fit a smoothing spline in Python, we use `LinearGAM()` function in the `pygam` library. We start with generating data as follows:


For $ x \in [0, 10]$:

$$
y = \sin(0.5x) \times \sin(x) + 0.1\epsilon
$$
where $\epsilon\sim N(0,1)$.

In [ ]:
np.random.seed(100)
x = np.linspace(0, 10, 30)
y = np.sin(0.5 * x) * np.sin(x) + 0.1 * np.random.randn(30)

plt.scatter(x, y)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In `pyGAM`, we can specify the spline terms by `s()`in the basis functional form. [Documentation](https://pygam.readthedocs.io/en/latest/api/api.html#spline-term).

In [ ]:
# Reshaping x to be a 2D array
X = x.reshape(-1, 1)

gam = LinearGAM(s(0, lam=0.6)).fit(X, y)
XX = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
plt.scatter(x, y)
plt.plot(XX, gam.predict(XX), label=rf"$\lambda$={0.6}")
plt.show()

In [ ]:
# Lambda values to try
lambda_values = [0, 0.1, 1, 10, 100, 100000]

# Plotting the original data
plt.scatter(X, y, facecolor="gray", label="Data")

# Fitting a model for lambda=0 and checking coefficients
gam_zero_lambda = LinearGAM(s(0, lam=0)).fit(X, y)

# Predicting on original X values for lambda=0
y_pred_zero_lambda = gam_zero_lambda.predict(X)
plt.plot(X, np.sin(0.5 * x) * np.sin(x), label="Original Y", color="black", ls=":", lw=2)

# Fitting a model and plotting for each non-zero lambda
for lam in lambda_values:
    gam = LinearGAM(s(0, lam=lam)).fit(X, y)
    XX = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
    plt.plot(XX, gam.predict(XX), label=rf"$\lambda$={lam}")

plt.title("Smoothing Spline Fits for Different λ Values")
plt.legend()
plt.show()

**Think About:** How would you measure bias-variance for this family?

### Generalized Additive Models ([GAMs](https://en.wikipedia.org/wiki/Generalized_additive_model))


GAMs extend the multiple linear regression model
$$
y_i = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + \cdots + \beta_p x_{ip} + \epsilon_i,
$$
to allow non-linear relationships between each feature and the response by replacing each linear compontent $\beta_j x_{ij}$ with a smooth non-linear function $f_j(x_{ij})$.
$$
g(x_i) = \beta_0 + f_1(x_{i1}) + f_2 (x_{i2}) + \cdots + f_p (x_{ip}) + \epsilon_i,
$$

$g(x_i)$ is indeed a linear combination of the smooth functions $f_j(x_{ij})$, but it's important to note that the "linearity" here refers to the linearity in the parameters (coefficients of the spline basis functions), not necessarily linearity in the relationship between predictors and the response variable. The model is "additive" because it sums up the effects of each predictor, allowing for flexible, non-linear relationships to be captured through the smooth functions $f_j$.

In [ ]:
# Dataset URL
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"

# Column names for the dataset
column_names = [
    "MPG",
    "Cylinders",
    "Displacement",
    "Horsepower",
    "Weight",
    "Acceleration",
    "Model Year",
    "Origin",
    "Car Name",
]

# Load the dataset
df = pd.read_csv(url, sep=r"\s+", names=column_names, na_values="?")

# Dropping rows with missing values for simplicity
df = df.dropna()

# Drop 'Car Name' column as it's not needed for this example
df = df.drop(columns=["Car Name"])

# Convert "Origin" into a categorical variable
df["Origin"] = df["Origin"].astype("category")

df.head()

In [ ]:
X = df.drop("MPG", axis=1).values
y = df["MPG"].values

In `pyGAM`, we can specify the basis functional form using terms:
- `l()`: linear terms
- `s()`: spline terms
- `f()`: factor terms
- `te()`: tensor products
- `intercept`: included by default

In the example below, we fit a spline term on feature 0 and feature 1, and a factor term on feature -1 (last column).

In [ ]:
# fit the model
gam = LinearGAM(s(feature=0, n_splines=20) + s(feature=1, n_splines=20) + f(feature=-1))
gam.fit(X, y)

In [ ]:
gam.summary()

**Think About:** How to choose the best $\lambda$?

Check [here](https://pygam.readthedocs.io/en/latest/api/lineargam.html) for the documentation and [here](https://pygam.readthedocs.io/en/latest/notebooks/quick_start.html) for the tutorial on **Automatic model tuning**. It selects the model with the lowest **generalized cross-validation (GCV)** score.

In [ ]:
lam = np.logspace(-3, 3, 11)
lams = [lam] * 3
gam.gridsearch(X, y, lam=lams)

In [ ]:
print(f"Optimal lambda: {gam.lam}")

#### GAMs for Classification
Recalling the usual logistic regression model
$$
\log(\frac{p(X)}{1-P(X)}) =  \beta_0 + \beta_1 X_{i1} + \beta_2 X_{i2} + \cdots + \beta_p X_{ip} + \epsilon_i,
$$
GAMs can also, analogously, extend the logistic regression model to capture non-linear relationships:
$$
\log(\frac{p(X)}{1-P(X)}) = \beta_0 + f_1(x_{i1}) + f_2 (x_{i2}) + \cdots + f_p (x_{ip}) + \epsilon_i,
$$

For the classification problem, we will use the `LogisticGAM `function from `pygam` package with the `breast_cancer` dataset. The target variable is whether the tumor of malignant (0) or benign (1), and the features are several measurements of the tumor.

In [ ]:
# URL for the Breast Cancer dataset (UCI Machine Learning Repository)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
# Column names for the dataset
column_names = ["ID", "Diagnosis"] + [f"Feature_{i}" for i in range(1, 31)]
df = pd.read_csv(url, header=None, names=column_names)

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the 'Diagnosis' column
# M stands for Malignant
# B stands for Benign
df["Diagnosis"] = label_encoder.fit_transform(df["Diagnosis"])


df.head()

In [ ]:
X = df.drop("Diagnosis", axis=1).values
y = df["Diagnosis"].values

# Fit a LogisticGAM with default settings as a starting point
gam = LogisticGAM(s(0) + s(1)).fit(X, y)

In [ ]:
gam.accuracy(X, y)

In [ ]:
gam.summary()

In [ ]:
# plotting
fig, axs = plt.subplots(1, 2, figsize=(14, 6))
titles = ["Feature 1", "Feature 2"]
for i, ax in enumerate(axs):
    # create a nice grid of X data
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=0.95)[1], c="r", ls="--")
    ax.set_ylim(-30, 30)
    ax.set_title(titles[i])

plt.show()

## Group Work Problems

---

**Problem 1:** Piecewise Constant Design Matrix

In class, we mentioned that one can perform piecewise constant regression by appropriately modifying the design matrix to a sparse 1, 0 matrix.

For the input vector $X$ and knots at $x = 3$ and $x = 6$:
$$
X=
\begin{bmatrix}
  1\\
  2\\
  4\\
  6\\
  7\\
  9
\end{bmatrix}
\longrightarrow
\begin{bmatrix}
  1 & 0 & 0 \\  
  1 & 0 & 0 \\  
  0 & 1 & 0 \\  
  0 & 0 & 1 \\  
  0 & 0 & 1 \\  
  0 & 0 & 1
\end{bmatrix}
$$
where the columns are obtained as $\mathbf{1}(X < 3), \mathbf{1}(3 \leq X < 6), \mathbf{1}(6 \leq X)$.

Implement a function `piecewise_constant_design_matrix` that takes in a vector `X` and a sorted array of `knots`, and returns the corresponding sparse feature matrix where each row has exactly one 1 indicating which interval the value falls into.

In [ ]:
def piecewise_constant_design_matrix(X: np.ndarray, knots: np.ndarray) -> np.ndarray:
    """
    Construct a design matrix for piecewise constant regression with disjoint bins.

    Parameters:
        X (np.ndarray): 1D array of predictor values.
        knots (np.ndarray): 1D array of knot positions (sorted).

    Returns:
        np.ndarray: Design matrix where each column corresponds to an interval
                    between consecutive knots, with an additional column for X < min(knots)
                    and another column for X >= max(knots).
    """
    # BEGIN SOLUTION
    # Strategy: Create indicator columns for each interval defined by the knots
    X = X.reshape(-1, 1)  # Ensure X is a column vector
    knots = np.sort(knots)  # Ensure knots are sorted

    # Define regions: First region before the first knot, intermediate intervals, and last region
    design_matrix = [(knots[0] > X).astype(int)]  # First region: X < first knot

    for idx in range(len(knots) - 1):
        # Intermediate bins: knots[idx] <= X < knots[idx+1]
        design_matrix.append(((knots[idx] <= X) & (knots[idx + 1] > X)).astype(int))

    design_matrix.append((knots[-1] <= X).astype(int))  # Last region: X >= last knot

    return np.column_stack(design_matrix)
    # END SOLUTION

In [ ]:
# Test assertions
# First, test with the simple example from the problem statement
X_simple = np.array([1, 2, 4, 6, 7, 9])
knots_simple = np.array([3, 6])
design_simple = piecewise_constant_design_matrix(X_simple, knots_simple)
expected_design = np.array(
    [
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
    ]
)
assert np.array_equal(design_simple, expected_design), "Simple example should match expected design"

# Random test with more samples
np.random.seed(503)
n_samples = 1000
upper = np.random.uniform(0, 100)
lower = np.random.uniform(0, upper)
x = np.random.uniform(lower, upper, n_samples)
X = x.reshape(-1, 1)

# Random knots between upper and lower
n_knots = 6
knots = np.random.uniform(lower, upper, n_knots)
knots = np.sort(knots)

# Generate feature matrix
X_design = piecewise_constant_design_matrix(X, knots)

# Visible test assertions
assert X_design.sum() == n_samples, f"Total sum should be {n_samples}, got {X_design.sum()}"
assert X_design.shape[0] == n_samples, f"Should have {n_samples} rows, got {X_design.shape[0]}"
assert (
    X_design.shape[1] == n_knots + 1
), f"Should have {n_knots + 1} columns, got {X_design.shape[1]}"
assert np.all(
    X_design.sum(axis=1) == 1
), "Each row should sum to 1 (exactly one indicator per sample)"
print("All tests passed!")

# BEGIN HIDDEN TESTS
expected_sum = np.array([204, 129, 76, 170, 123, 270, 28])
assert np.array_equal(X_design.sum(axis=0), expected_sum), "Column sums should match expected"

# Test edge case with values exactly on knots
X_knot = np.array([3.0, 6.0])
design_knot = piecewise_constant_design_matrix(X_knot, knots_simple)
assert design_knot[0, 1] == 1, "Value exactly on first knot should be in middle interval"
assert design_knot[1, 2] == 1, "Value exactly on last knot should be in last interval"
# END HIDDEN TESTS

---

**Problem 2:** Piecewise Constant Regression

Create a function `piecewise_constant_regression` that takes in `X`, `y`, and a list of `knots` and performs piecewise constant regression using the design matrix from Problem 1.

The function should return a fitted `LinearRegression` model (with `fit_intercept=False` since the design matrix already captures the constant pieces).

In [ ]:
def piecewise_constant_regression(
    X: np.ndarray, y: np.ndarray, knots: np.ndarray
) -> LinearRegression:
    """
    Perform piecewise constant regression using a design matrix.

    Parameters:
        X (np.ndarray): 1D array of predictor values.
        y (np.ndarray): 1D array of target values.
        knots (np.ndarray): 1D array of knot positions (sorted).

    Returns:
        LinearRegression: Fitted LinearRegression model for piecewise constant regression.
    """
    # BEGIN SOLUTION
    # Strategy: Transform X to design matrix then fit linear regression without intercept
    X_design = piecewise_constant_design_matrix(X, knots)
    model = LinearRegression(fit_intercept=False)
    model.fit(X_design, y)
    return model
    # END SOLUTION

In [ ]:
# Test assertions
np.random.seed(503)
n_samples = 300
x = np.random.uniform(0, 10, n_samples)
y = 2 * np.sin(0.5 * x) + np.cos(x) + np.sqrt(0.1) * np.random.randn(n_samples)
X = x.reshape(-1, 1)

knots = np.array([2, 5, 8])

# Fit the piecewise constant model
model = piecewise_constant_regression(X, y, knots)

# Verify model properties
assert hasattr(model, "coef_"), "Model should have coefficients"
assert model.coef_.shape[0] == len(knots) + 1, "Should have one coefficient per interval"

# Plot the piecewise constant fit
X_fit = np.linspace(0, 10, 100).reshape(-1, 1)
X_fit_design = piecewise_constant_design_matrix(X_fit, knots)
y_pred = model.predict(X_fit_design)

# Compute sum of squared residuals
y_pred_train = model.predict(piecewise_constant_design_matrix(X, knots))
ssr = np.mean((y - y_pred_train) ** 2)

# Scatter plot data
plt.scatter(X, y, color="gray", alpha=0.2, label="Data")
plt.plot(X_fit, model.predict(X_fit_design), label="Piecewise Constant Fit", color="red")
plt.text(
    0.95,
    0.95,
    f"MSE: {ssr:.2f}",
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment="top",
    horizontalalignment="right",
)
plt.legend()
plt.show()

print("All tests passed!")

# BEGIN HIDDEN TESTS
# Verify the coefficients are close to expected means in each interval
expected_coefs = np.array([1.67, 1.96, -0.47, 1.44])
assert np.allclose(model.coef_, expected_coefs, atol=0.1), "Coefficients should match expected"

# Test that fit_intercept is False
assert model.fit_intercept is False, "Model should not have intercept"

# Test MSE is reasonable
assert ssr < 1.0, "MSE should be less than 1.0"
# END HIDDEN TESTS

---

**Problem 3:** Loading NHANES Data

Import the NHANES datasets and prepare them for analysis:

(a) Keep the HDL in mg/dL as your target variable. Also, use the following features for predictive purposes: Gender, Age, Weight, Height, BMI, WaistSize, Household Size, and Ethnicity. You may need to refer to the docs to figure out their variable names:
- [HDL_L](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/HDL_L.htm)
- [DEMO_L](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.htm)
- [BMX_L](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.htm)

(b) Rename the variable names to be human-readable in Python variable style (e.g., `BMXWT` becomes `Weight`).

(c) Drop all missing values.

Store the final DataFrame in a variable called `my_df`.

In [ ]:
# BEGIN SOLUTION
# Strategy: Load SAS files, merge on SEQN, select and rename columns, drop NAs
bmx_df = pd.read_sas("data/NHANES/BMX_L.xpt")
demo_df = pd.read_sas("data/NHANES/DEMO_L.xpt")
hdl_df = pd.read_sas("data/NHANES/HDL_L.xpt")

# Inner join on SEQN (sequence number)
df = pd.merge(hdl_df, bmx_df, on="SEQN", how="inner")
df = pd.merge(df, demo_df, on="SEQN", how="inner")

# Select and rename columns
selected_columns = [
    "LBDHDD",
    "RIAGENDR",
    "RIDAGEYR",
    "BMXWT",
    "BMXHT",
    "DMDHHSIZ",
    "BMXBMI",
    "BMXWAIST",
    "RIDRETH1",
]

my_df = df[selected_columns].copy()
my_df = my_df.rename(
    columns={
        "LBDHDD": "HDL",
        "RIAGENDR": "Gender",
        "RIDAGEYR": "Age",
        "BMXWT": "Weight",
        "BMXHT": "Height",
        "BMXBMI": "BMI",
        "BMXWAIST": "WaistSize",
        "DMDHHSIZ": "HouseholdSize",
        "RIDRETH1": "Ethnicity",
    }
)
my_df = my_df.dropna()
# END SOLUTION

In [ ]:
# Test assertions
assert my_df.shape == (6589, 9), f"Expected shape (6589, 9), got {my_df.shape}"
assert "HDL" in my_df.columns, "DataFrame should have HDL column"
print("All tests passed!")

# BEGIN HIDDEN TESTS
expected_columns = [
    "HDL",
    "Gender",
    "Age",
    "Weight",
    "Height",
    "BMI",
    "WaistSize",
    "HouseholdSize",
    "Ethnicity",
]
assert list(my_df.columns) == expected_columns, "Columns should match expected names"
assert my_df.isna().sum().sum() == 0, "No missing values should remain"
assert my_df["HDL"].dtype == np.float64, "HDL should be float"
# END HIDDEN TESTS

---

**Problem 4:** Spline Pipeline

Create a sklearn `Pipeline` called `pipeline` that performs the following steps:

1. Create degree 4 spline features with 8 knots using `SplineTransformer`
2. Apply `StandardScaler` for feature scaling
3. Fit a `LinearRegression` model

Fit the pipeline using BMI as the predictor (`X = my_df[['BMI']]`) and HDL as the target (`y = my_df[['HDL']]`).

In [ ]:
# BEGIN SOLUTION
# Strategy: Chain SplineTransformer, StandardScaler, and LinearRegression in a Pipeline
pipeline = Pipeline(
    [
        ("spline_features", SplineTransformer(degree=4, n_knots=8)),
        ("scaler", StandardScaler()),
        ("regressor", LinearRegression()),
    ]
)

X = my_df[["BMI"]]
y = my_df[["HDL"]]
pipeline.fit(X, y)
# END SOLUTION

In [ ]:
# Test assertions
fig, axs = plt.subplots(1, 2, figsize=(14, 6))
titles = ["BMI vs. HDL", "Residuals of BMI vs. HDL"]
axs[0].scatter(X, y, color="gray", alpha=0.1, label="Data")
x_linspace = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
axs[0].plot(x_linspace, pipeline.predict(x_linspace), label="Spline Fit", color="red")
axs[0].set_xlabel("BMI")
axs[0].set_ylabel("HDL")
axs[0].set_title(titles[0])
axs[0].legend()
axs[1].scatter(my_df["BMI"], pipeline.predict(X) - y, color="gray", alpha=0.1)
axs[1].set_xlabel("BMI")
axs[1].set_ylabel("Residuals")
axs[1].set_title(titles[1])
plt.show()

# Verify pipeline structure
assert len(pipeline.steps) == 3, "Pipeline should have 3 steps"
assert pipeline.named_steps["spline_features"].degree == 4, "Spline degree should be 4"
assert pipeline.named_steps["spline_features"].n_knots == 8, "Number of knots should be 8"
print(f"Linear Regression Coefficients: {pipeline.named_steps['regressor'].coef_[0]}")
print("All tests passed!")

# BEGIN HIDDEN TESTS
# Verify the pipeline components
assert "spline_features" in pipeline.named_steps, "Should have spline_features step"
assert "scaler" in pipeline.named_steps, "Should have scaler step"
assert "regressor" in pipeline.named_steps, "Should have regressor step"

# Verify coefficient shape (degree 4 + n_knots 8 = 4 + 8 - 1 = 11 features)
n_features = pipeline.named_steps["regressor"].coef_.shape[1]
assert n_features == 11, f"Expected 11 features, got {n_features}"
# END HIDDEN TESTS

---

**Problem 5:** GridSearchCV for Spline Parameters

We want to decide how many knots and what degree of spline to use. Working with all the predictive features (everything except HDL), use [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find the optimal parameter values.

Use the following settings:
- 10-fold cross-validation
- `neg_mean_squared_error` as the scoring metric
- Search over these parameters:
  - `degree`: 2, 3, 4
  - `n_knots`: 4, 8, 16, 32

Store your fitted `GridSearchCV` object in a variable called `grid_search`.

In [ ]:
# BEGIN SOLUTION
# Strategy: Set up pipeline with SplineTransformer and use GridSearchCV to find optimal parameters
X = my_df.drop("HDL", axis=1)
y = my_df["HDL"]

pipeline = Pipeline(
    [
        ("spline_features", SplineTransformer()),
        ("scaler", StandardScaler()),
        ("regressor", LinearRegression()),
    ]
)

# Define parameter grid
param_grid = {
    "spline_features__n_knots": [4, 8, 16, 32],
    "spline_features__degree": [2, 3, 4],
}

# Perform GridSearchCV using neg_mean_squared_error
grid_search = GridSearchCV(
    pipeline, param_grid, cv=10, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1
)
grid_search.fit(X, y)
# END SOLUTION

In [ ]:
# Test assertions
expected = {"spline_features__degree": 2, "spline_features__n_knots": 4}
assert grid_search.best_params_ == expected, f"Expected {expected}, got {grid_search.best_params_}"
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")
print("All tests passed!")

# BEGIN HIDDEN TESTS
# Verify GridSearchCV settings
assert grid_search.cv == 10, "Should use 10-fold CV"
assert grid_search.scoring == "neg_mean_squared_error", "Should use neg_mean_squared_error"

# Verify coefficient shape matches expected
n_features = 8  # number of predictors
n_spline_features = n_features * (4 + 2 - 1)  # n_knots=4, degree=2
assert grid_search.best_estimator_.named_steps["regressor"].coef_.shape == (
    n_spline_features,
), f"Coefficient shape should be ({n_spline_features},)"
# END HIDDEN TESTS

---

**Problem 6:** Loading the Default Dataset

Import the `default` dataset from `pygam.datasets`. See [documentation](https://pygam.readthedocs.io/en/latest/) and [GitHub](https://github.com/dswah/pyGAM/blob/master/pygam/datasets/load_datasets.py).

Create an 80-20 train-test split with `random_state=503` and store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [ ]:
# BEGIN SOLUTION
# Strategy: Load default dataset and create train-test split
X, y = default(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=503)
# END SOLUTION

In [ ]:
# Test assertions
assert X_train.shape[0] == 8000, f"Expected 8000 training samples, got {X_train.shape[0]}"
assert X_test.shape[0] == 2000, f"Expected 2000 test samples, got {X_test.shape[0]}"
assert X_train.shape[1] == 3, "Features should have 3 columns"
print("All tests passed!")

# BEGIN HIDDEN TESTS
assert y_train.shape[0] == 8000, "y_train should have 8000 samples"
assert y_test.shape[0] == 2000, "y_test should have 2000 samples"
assert X.shape == (10000, 3), "Full X should be (10000, 3)"
assert set(y) == {0, 1}, "y should be binary"
# END HIDDEN TESTS

---

**Problem 7:** Logistic GAM with Gridsearch

Fit a `LogisticGAM` model with:
- A factor term `f()` on the first variable (student status, index 0)
- Spline terms `s()` on the remaining variables (balance and income, indices 1 and 2)

Use the `gridsearch` method to find optimal lambda values. You can use the default lambda grid which uses `lam=np.logspace(-3, 3, 11)`.

Store your fitted GAM in a variable called `gam`.

In [ ]:
# BEGIN SOLUTION
# Strategy: Use LogisticGAM with factor term on first variable and splines on the rest
# Use gridsearch to find optimal lambda values
gam = LogisticGAM(f(0) + s(1) + s(2)).gridsearch(X_train, y_train)
# END SOLUTION

In [ ]:
# Test assertions
# Verify the model has expected structure
assert hasattr(gam, "lam"), "GAM should have lambda parameters after gridsearch"
assert len(gam.terms) == 4, "Should have 3 terms plus intercept"
print(f"Optimal lambdas: {gam.lam}")

# Plot partial dependence
fig, axs = plt.subplots(1, 3, figsize=(14, 4))
titles = ["student", "balance", "income"]

for term_idx, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=term_idx)
    pdep, confi = gam.partial_dependence(term=term_idx, width=0.95)
    ax.plot(XX[:, term_idx], pdep)
    ax.plot(XX[:, term_idx], confi, c="r", ls="--")
    ax.set_title(titles[term_idx])

plt.tight_layout()
plt.show()

print("All tests passed!")

# BEGIN HIDDEN TESTS
# Verify the terms are correctly specified
assert gam.terms[0].isintercept is False, "First term should not be intercept"
assert gam.accuracy(X_test, y_test) > 0.9, "Model accuracy on test set should be > 90%"

# Verify gridsearch was performed
train_accuracy = gam.accuracy(X_train, y_train)
assert train_accuracy > 0.9, f"Training accuracy {train_accuracy} should be > 0.9"
# END HIDDEN TESTS

---

**Problem 8:** Interpreting Partial Dependence Plots (free response)

Examine the partial dependence plots from Problem 7 and comment on what they reveal about the relationship between each predictor and the probability of default. Consider:

- What is the effect of student status on default probability?
- How does balance affect the log-odds of default?
- What is the relationship between income and default probability?

> BEGIN SOLUTION

The partial dependence plots reveal the following relationships:

1. **Student status**: Being a student (value 1) is associated with a higher log-odds of default compared to non-students (value 0). This suggests that students have a higher probability of defaulting on their credit, possibly due to lower and less stable income during their studies.

2. **Balance**: There is a strong positive, roughly linear relationship between credit card balance and the log-odds of default. As balance increases, the probability of default increases substantially. This makes intuitive sense as higher balances indicate more debt and potentially greater financial strain.

3. **Income**: Income shows a relatively flat relationship with default probability, with wide confidence intervals indicating uncertainty. This suggests that income, after controlling for balance and student status, has little direct effect on the probability of default. The effect of income may already be captured through its correlation with balance levels.
> END SOLUTION
